## Import libraries

In [ ]:
import requests
from io import BytesIO
from nltk import sent_tokenize
from textwrap import wrap
from PIL import Image, ImageDraw, ImageFont, ImageOps

import openai
import gradio as gr

import warnings
warnings.filterwarnings("ignore")

## Setting Open API key

In [ ]:
openai.api_key = "enter your key here"

## Function to get story from ChatGPT

In [ ]:
def chatgpt_api(genre, words):
    # Logging
    print(f"Generating story for Genre: {genre}.\tWords: {words}")

    wordsList = ", ".join(words)

    messages = [
    {"role": "system", "content": "You are a helpful assistant."}]
    
    if words and genre:
        messages.append(
            {"role": "user", "content": f"In 100 words, write a story with the words: {wordsList}. It should be in the genre of {genre}"},
        )
        
        chat_completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
    
    story = chat_completion.choices[0].message.content
    return story

## Function to generate images using Dalle

In [ ]:
def dall_e_api(dalle_prompt):
    dalle_response = openai.Image.create(
            prompt = dalle_prompt,
            size="512x512"
        )
    image_url = dalle_response['data'][0]['url']

    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    return image

## Function to put text in generated image

In [ ]:
def write_line(image, line, v_offset):
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("arialbd.ttf", 18)
    _, _, w, _ = draw.textbbox((0, 0), line, font=font)
    draw.text(
        xy=((512-w)/2, 490-v_offset),
        text=line,
        font=font,
        fill=(255,255,255),
        align="center",
        stroke_width=1,
        stroke_fill=(0,0,0))
    return image


def write_text(image, text):
    lines = wrap(text, width=53)
    for i, line in enumerate(lines):
        v_offset = (len(lines)- i) * 21
        image = write_line(image, line, v_offset)
    
    return image

## Function to get story images from story

In [ ]:
def get_story_images(story, genre):
    sentenceList = sent_tokenize(story)
    
    images = []
    for sentence in sentenceList:
        # Logging
        print(f"Generating image for: {sentence}")

        dalle_prompt = f'{sentence} Genre:"{genre}"'
        image = dall_e_api(dalle_prompt)
        images.append(write_text(image, sentence))

    return images

## Function to make comic strip

In [ ]:
def get_comic(images):
    # Logging
    print("Creating comic")
    
    rows = len(images)
    height = rows * 530
    comic = Image.new('RGB', (540, height), "white")

    y = 7
    for row in range(rows):
        img = ImageOps.expand(images[row], border=2, fill='black')
        comic.paste(img, (12, y))
        y += 530

    return comic


def get_default_comic():
    default = Image.new('RGB', (512, 512), "white")
    default = write_text(default, "Dalle cannot generate images for the story. Please try different prompts.")
    return default

## Function to take user input and generate story

In [ ]:
def generate_story(genre, *wordList):
    print(wordList)
    words = []
    for word in wordList:
        if word == "":
            continue
        words.append(word)
    print(words)
    story = chatgpt_api(genre, words)

    return story

def generate_comic(story, genre):
    try:
        images = get_story_images(story, genre)
        comic = get_comic(images)
    except Exception as ex:
        comic = get_default_comic()

        # Logging error message
        if hasattr(ex, 'message'):
            print(ex.message)
        else:
            print(ex)

    return comic

# Gradio Interface

In [ ]:
with gr.Blocks() as app:
    gr.Markdown('<h1 style="text-align: center;">Story and Comic Generator</h1>')
    gr.Markdown('<p style="text-align: center;">using the Open AI API</p>')

    # Giving input to generate story
    gr.Markdown("## Please enter the genre for the story below")
    genre = gr.Textbox(label="Genre")

    gr.Markdown("## Please enter the keywords that must be used below")
    words = []
    counter = 1
    for i in range(2):
        with gr.Row():
            for j in range(3):
                words.append(gr.Textbox(label=f"Word {counter}"))
                counter += 1

    gr.Markdown("<hr>")

    # Generating Story
    story_btn = gr.Button("Generate Story")
    story = gr.Textbox(label="Generated Story", interactive=False)
    story_btn.click(fn=generate_story,
              inputs=[genre,
                      words[0],
                      words[1],
                      words[2],
                      words[3],
                      words[4],
                      words[5]],
              outputs=story)
    
    gr.Markdown("<hr>")

    # Generating Comic
    gr.Markdown("Please note that there are restrictions on the Dalle API limiting it to 5 images per minute. It also follows certain security restrictions which stop it from generating images related to certain topics. If the Comic fails to generate due to any reason, you could try using different prompts or generate comics for a story that is 5 sentences or lesser")
    comic_btn = gr.Button("Generate Comic")
    gr.Markdown("Comic generation may take about 2-3 minutes. Please be patient")
    comic = gr.Image(label="Generated Comic")

    comic_btn.click(fn=generate_comic,
              inputs=[story, genre],
              outputs=comic)

app.title = "Story and Comic Generator"
app.launch()